In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import TextStreamer, GenerationConfig

model_name='davidkim205/komt-mistral-7b-v1'
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", low_cpu_mem_usage=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)
streamer = TextStreamer(tokenizer)

def gen(x):
    generation_config = GenerationConfig(
        temperature=0.8,
        top_p=0.8,
        top_k=100,
        max_new_tokens=512,
        early_stopping=True,
        do_sample=True,
    )
    q = f"[INST]{x} [/INST]"
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"\n\n### Response: "
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [2]:
import platform
import os
import pandas as pd


TEST_NAME = "llm1"

# 데이터 관련
os_name = platform.system()
if os_name == 'Windows':
    PRE_PATH = ''
elif os_name == 'Linux':
    PRE_PATH = '/kkh/'
elif os_name == 'Darwin': # 맥
    PRE_PATH = '/kkh/'
DATA_PATH = PRE_PATH + "data/" # 대회에서 제공한 데이터
OUTPUT_PATH = PRE_PATH + "output/" # 모델의 출력 값
CHECKPOINT_PATH = PRE_PATH + "checkpoint/" # 모델의 최종 출력 값
PREDICTION_PATH = PRE_PATH + "prediction/" # 최종 예측 값
LOG_PATH = PRE_PATH + "log/"
TRAIN_PATH = DATA_PATH + "train_new2.csv"
VALID_PATH = DATA_PATH + "dev_new2.csv"
TEST_PATH = DATA_PATH + "test.csv"
SUBMIT_PATH = PREDICTION_PATH + "submission_" + TEST_NAME + ".csv"

train_df = pd.read_csv(os.path.join(TRAIN_PATH))
val_df = pd.read_csv(os.path.join(VALID_PATH))
test_df = pd.read_csv(os.path.join(TEST_PATH))

sampled_df = train_df.sample(n=5, random_state=42)  # random_state는 결과를 재현 가능하게 합니다.
combined_string = '\n'.join(sampled_df.apply(lambda row: f"{row['fname']}, {row['dialogue']}, {row['summary']}, {row['topic']}", axis=1).tolist())
print(combined_string)

train_1057, #Person1#: 그래도 저는 이 벽지를 원하지 않았어요. 그리고 프로그램을 열려고 할 때마다 여장을 한 부시 대통령만 보여요. 
#Person2#: 그 끔찍한 바이러스에 대해 들었어요! 자동으로 주소록에 있는 모든 사람에게 첨부 파일로 이메일을 보내요.
#Person1#: 글쎄요, 이미 제 컴퓨터에 올라와 있어요. 어떻게 없앨 수 있죠?
#Person3#: 다음 선거에서 투표로 없애세요.
#Person1#: 이제 그만하고 진지하게 대답해줘요!
#Person3#: IT 부서에게 바이러스 제거 프로그램을 실행하도록 요청하세요. 2005년까지 문제가 해결될 것이라고 생각해요., #Person1#의 컴퓨터에 끔찍한 바이러스가 걸렸고, #Person2#는 IT 부서에게 바이러스 제거 프로그램을 실행하도록 제안했다., 컴퓨터 바이러스
train_10667, #Person1#: 우리 회사에 관심이 있는 이유는 무엇인가요?
#Person2#: 귀사는 세계적으로 유명합니다. 그래서 개인적인 발전을 위한 더 나은 기회가 있을 것이라고 생각합니다.
#Person1#: 맞습니다. 이 직무에 필요한 자격을 갖추고 있다고 생각하나요?
#Person2#: 그럼요. 제 자격은 귀사의 직무 설명에 부합합니다.
#Person1#: 하지만 이 직무에 대한 경험이 없습니다.
#Person2#: 그건 사실입니다. 하지만 배우는 것에 열정이 있고, 빠르게 배울 수 있습니다.
#Person1#: 추가 교육이 필요하다면, 교육비를 아시나요?
#Person2#: 아니요. 월급은 얼마인가요?
#Person1#: 월 800위안입니다.
#Person2#: 내일 답변을 드려도 괜찮을까요?
#Person1#: 예. 물론입니다., #Person2#는 세계적으로 유명한 #Person1#의 회사에 관심이 있으며, 경험이 없음에도 불구하고 #Person2#는 자신이 직무에 필요한 자격을 갖추고 있다고 생각합니다., 직무 인터뷰
train_5261, #Person1#: 데이비드, 우리가 여기 앉아

In [3]:
summary_list = []
for index, row in test_df.iterrows():
    print('== start ===================================')
    row['summary'] = '"' + gen(f"\n\n\
        - 키워드 {row['keyword']}를 꼭 포함해서 아래 대화문을 요약해주세요.\n\
        - 요약문은 최대 두 문장으로 표현해주고, 더불어, 요약문의 len() 길이는 101을 넘지 않도록 해주세요.\n\
        - 요약문은 3인칭 관찰자가 설명하듯이 적어주세요.\n\
        - 사람 이름 대신에, #Person1# 이나 #Person2# 와 같은 키워드 형태로 적어주세요.\n\
    \n{row['dialogue']}") + '"'
    print(f"@ row['fname']: {row['fname']}")
    print(f"@ row['summary']: {row['summary']}")
    print('== end ===================================')
    summary_list.append(row['summary'])

/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (4006 > 1024). Running this sequence through the model will result in indexing errors


<s> [INST]train_1057, #Person1#: 그래도 저는 이 벽지를 원하지 않았어요. 그리고 프로그램을 열려고 할 때마다 여장을 한 부시 대통령만 보여요. 
#Person2#: 그 끔찍한 바이러스에 대해 들었어요! 자동으로 주소록에 있는 모든 사람에게 첨부 파일로 이메일을 보내요.
#Person1#: 글쎄요, 이미 제 컴퓨터에 올라와 있어요. 어떻게 없앨 수 있죠?
#Person3#: 다음 선거에서 투표로 없애세요.
#Person1#: 이제 그만하고 진지하게 대답해줘요!
#Person3#: IT 부서에게 바이러스 제거 프로그램을 실행하도록 요청하세요. 2005년까지 문제가 해결될 것이라고 생각해요., #Person1#의 컴퓨터에 끔찍한 바이러스가 걸렸고, #Person2#는 IT 부서에게 바이러스 제거 프로그램을 실행하도록 제안했다., 컴퓨터 바이러스
train_10667, #Person1#: 우리 회사에 관심이 있는 이유는 무엇인가요?
#Person2#: 귀사는 세계적으로 유명합니다. 그래서 개인적인 발전을 위한 더 나은 기회가 있을 것이라고 생각합니다.
#Person1#: 맞습니다. 이 직무에 필요한 자격을 갖추고 있다고 생각하나요?
#Person2#: 그럼요. 제 자격은 귀사의 직무 설명에 부합합니다.
#Person1#: 하지만 이 직무에 대한 경험이 없습니다.
#Person2#: 그건 사실입니다. 하지만 배우는 것에 열정이 있고, 빠르게 배울 수 있습니다.
#Person1#: 추가 교육이 필요하다면, 교육비를 아시나요?
#Person2#: 아니요. 월급은 얼마인가요?
#Person1#: 월 800위안입니다.
#Person2#: 내일 답변을 드려도 괜찮을까요?
#Person1#: 예. 물론입니다., #Person2#는 세계적으로 유명한 #Person1#의 회사에 관심이 있으며, 경험이 없음에도 불구하고 #Person2#는 자신이 직무에 필요한 자격을 갖추고 있다고 생각합니다., 직무 인터뷰
train_5261, #Person1#: 데이비드,

/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


[/INST] 핵심 키워드: 대화문, 요약문, 내부 메모, 직원들, 외부 통신, 메시지 프로그램, 직무 정지, 해고

요약문: 실장님은 모든 직원들에게 내부 메모로 전달되어야 하는 내용을 전달하였습니다. 이 메모는 모든 직원에게 적용되며, 내부와 외부 통신에 대한 제한이 있습니다. 직원들은 메시지 프로그램을 사용하지 못하고, 두 번의 위반 시 직무 정지에 처해지며, 해고될 수도 있습니다. 직원들은 부서장에게 직접 문의하여 질문이 있다면 해결해 주기를 바랍니다.</s>
@ row['fname']: test_0
@ row['dialogue']: #Person1#: 더슨 씨, 받아쓰기 좀 해주세요. 
#Person2#: 네, 실장님...
#Person1#: 이것은 오늘 오후까지 모든 직원에게 내부 메모로 전달되어야 합니다. 준비되셨나요?
#Person2#: 네, 실장님. 시작하셔도 됩니다.
#Person1#: 모든 직원들에게 주의하라... 즉시 효력을 발휘하여, 모든 사무실 통신은 이메일 통신과 공식 메모로 제한됩니다. 근무 시간 동안 직원들이 즉시 메시지 프로그램을 사용하는 것은 엄격히 금지됩니다.
#Person2#: 실장님, 이것은 내부 통신에만 적용되는 건가요? 아니면 외부 통신에도 제한이 되는 건가요?
#Person1#: 이것은 모든 통신에 적용되어야 합니다, 이 사무실 내의 직원들 사이뿐만 아니라 외부 통신에도 마찬가지입니다.
#Person2#: 하지만 실장님, 많은 직원들이 고객과 소통하기 위해 즉시 메시지를 사용하고 있습니다.
#Person1#: 그들은 그들의 의사소통 방법을 바꾸어야만 합니다. 이 사무실에서 누구도 즉시 메시지를 사용하지 않기를 원합니다. 너무 많은 시간을 낭비하게 됩니다! 이제, 메모를 계속해주세요. 우리가 어디까지 했나요?
#Person2#: 이것은 내부와 외부 통신에 적용됩니다.
#Person1#: 그렇습니다. 즉시 메시지를 계속 사용하는 어떤 직원이라도 먼저 경고를 받고 직무 정지에 처해질 것입니다. 두 번째 위반 시에는 직원은

/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


...

[/INST] 핵심 키워드: 연락, 전화, 이메일, 결정, 시간, 감사

요약문: 연락 방법은 전화나 이메일로 가능하며, 결정은 한 주 정도 시간이 필요하다. 감사의 말씀을 전하며 연락주시면 된다.</s>
@ row['fname']: test_123
@ row['dialogue']: #Person1#: 우리는 어떻게 연락을 드릴 수 있을까요?
#Person2#: 전화나 이메일로 연락주시면 됩니다. 언제쯤 결정을 내려주실 수 있나요?
#Person1#: 대략 한 주 정도입니다.
#Person2#: 시간을 내주셔서 감사합니다. 그때 뵙겠습니다.
@ row['summary']: "<s> [INST]train_1057, #Person1#: 그래도 저는 이 벽지를 원하지 않았어요. 그리고 프로그램을 열려고 할 때마다 여장을 한 부시 대통령만 보여요. 
#Person2#: 그 끔찍한 바이러스에 대해 들었어요! 자동으로 주소록에 있는 모든 사람에게 첨부 파일로 이메일을 보내요.
#Person1#: 글쎄요, 이미 제 컴퓨터에 올라와 있어요. 어떻게 없앨 수 있죠?
#Person3#: 다음 선거에서 투표로 없애세요.
#Person1#: 이제 그만하고 진지하게 대답해줘요!
#Person3#: IT 부서에게 바이러스 제거 프로그램을 실행하도록 요청하세요. 2005년까지 문제가 해결될 것이라고 생각해요., #Person1#의 컴퓨터에 끔찍한 바이러스가 걸렸고, #Person2#는 IT 부서에게 바이러스 제거 프로그램을 실행하도록 제안했다., 컴퓨터 바이러스
train_10667, #Person1#: 우리 회사에 관심이 있는 이유는 무엇인가요?
#Person2#: 귀사는 세계적으로 유명합니다. 그래서 개인적인 발전을 위한 더 나은 기회가 있을 것이라고 생각합니다.
#Person1#: 맞습니다. 이 직무에 필요한 자격을 갖추고 있다고 생각하나요?
#Person2#: 그럼요. 제 자격은 귀사의 직무 설명에 부합합니다.
#Person1#: 하지만 이 직무에 대한 경험이 없습니다.
#Per

/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


[/INST] 핵심 키워드: 중국 와인, 무탄, 베이징 오리구이

요약문: #Person1#는 #Person2#에게 중국 와인을 추천해줄 수 있는지 물어보고, #Person2#는 무탄과 함께 전형적인 중국 요리인 베이징 오리구이를 추천해줍니다.</s>
@ row['fname']: test_124
@ row['dialogue']: #Person1#: 중국 와인을 추천해주실 수 있나요?
#Person2#: 무탄은 어떠세요?
#Person1#: 좋아요! 무탄과 함께 어떤 것이 나오나요?
#Person2#: 네, 전형적인 중국 요리를 주문하실 수 있습니다. 예를 들어, 베이징 오리구이.
#Person1#: 좋아 보이네요.
#Person2#: 네, 우리 호텔의 최고의 메뉴입니다.
#Person1#: 정말요? 한번 시도해볼게요.
#Person2#: 감사합니다. 잠시만 기다려주세요.
@ row['summary']: "<s> [INST]train_1057, #Person1#: 그래도 저는 이 벽지를 원하지 않았어요. 그리고 프로그램을 열려고 할 때마다 여장을 한 부시 대통령만 보여요. 
#Person2#: 그 끔찍한 바이러스에 대해 들었어요! 자동으로 주소록에 있는 모든 사람에게 첨부 파일로 이메일을 보내요.
#Person1#: 글쎄요, 이미 제 컴퓨터에 올라와 있어요. 어떻게 없앨 수 있죠?
#Person3#: 다음 선거에서 투표로 없애세요.
#Person1#: 이제 그만하고 진지하게 대답해줘요!
#Person3#: IT 부서에게 바이러스 제거 프로그램을 실행하도록 요청하세요. 2005년까지 문제가 해결될 것이라고 생각해요., #Person1#의 컴퓨터에 끔찍한 바이러스가 걸렸고, #Person2#는 IT 부서에게 바이러스 제거 프로그램을 실행하도록 제안했다., 컴퓨터 바이러스
train_10667, #Person1#: 우리 회사에 관심이 있는 이유는 무엇인가요?
#Person2#: 귀사는 세계적으로 유명합니다. 그래서 개인적인 발전을 위한 더 나은 기회가 있을 것이라고 

/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/envs/nlp/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 460.00 MiB. GPU 